In [4]:
API_KEY = ''

In [1]:
# We'll start this curl command for later
! curl "https://wagon-public-datasets.s3.amazonaws.com/deep_learning_datasets/llama-7b.ggmlv3.q4_1.bin" > "llama.bin"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4017M  100 4017M    0     0  9499k      0  0:07:13  0:07:13 --:--:-- 10.9M12k      0  0:07:12  0:00:31  0:06:41 10.5M  0     0  9614k      0  0:07:07  0:00:35  0:06:32 10.3MM    0     0  9404k      0  0:07:17  0:00:38  0:06:39 8192k  0:06:36 8984k  0  9254k      0  0:07:24  0:00:45  0:06:39 7086k19k      0  0:07:41  0:00:50  0:06:51 5845k   0  0:07:50  0:01:08  0:06:42 7570k:06:45 6873k7:50  0:01:35  0:06:15 9161k 26 1052M    0     0  9057k      0  0:07:34  0:01:58  0:05:36 10.9M  0     0  9201k      0  0:07:27  0:02:05  0:05:22 11.2M245k      0  0:07:24  0:02:14  0:05:10 9644k   0  0:07:13  0:02:36  0:04:37 10.9M17M   42 1723M    0     0  9331k      0  0:07:20  0:03:09  0:04:11 7757k6 4017M   46 1883M    0     0  9229k      0  0:07:25  0:03:28  0:03:57 8558k    0     0  9057k      0  0:07:34  0:05:45  0:01:49 10.9M 0:01:46 10

In [14]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.chains.question_answering import load_qa_chain

/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/langchain/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
from langchain.document_loaders import WikipediaLoader
 
# Load content from Wikipedia using WikipediaLoader
loader = WikipediaLoader("Machine_learning")
document = loader.load()

In [16]:
document

[Document(page_content='Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can effectively generalize and thus perform tasks without explicit instructions. Recently, generative artificial neural networks have been able to surpass many previous approaches in performance. Machine learning approaches have been applied to large language models, computer vision, speech recognition, email filtering, agriculture and medicine, where it is too costly to develop algorithms to perform the needed tasks.The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis through unsupervised learning.ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an

In [17]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)

In [18]:
docs=text_splitter.split_documents(document)
len(docs)

264

In [19]:
from langchain.vectorstores import Chroma


#get ada-002 embeddings
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

persist_directory = 'db/chroma/'

# Create the vector store
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

In [20]:
vectordb

In [21]:
docs=vectordb.similarity_search('what is overfitting?', k=2)


In [9]:
docs

[Document(page_content='the implementer is less likely to experience overfitting.', metadata={'source': 'https://en.wikipedia.org/wiki/Support_vector_machine', 'summary': 'In machine learning, support vector machines (SVMs, also support vector networks) are supervised learning models with associated learning algorithms that analyze data for classification and regression analysis. Developed at AT&T Bell Laboratories by Vladimir Vapnik with colleagues (Boser et al., 1992, Guyon et al., 1993, Cortes and Vapnik, 1995, Vapnik et al., 1997) SVMs are one of the most robust prediction methods, being based on statistical learning frameworks or VC theory proposed by Vapnik (1982, 1995) and Chervonenkis (1974). Given a set of training examples, each marked as belonging to one of two categories, an SVM training algorithm builds a model that assigns new examples to one category or the other, making it a non-probabilistic binary linear classifier (although methods such as Platt scaling exist to use 

In [36]:
from llama_cpp import Llama
llm = Llama(model_path="llama.bin")

llama.cpp: loading model from llama.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 3 (mostly Q4_1)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.07 MB
llama_model_load_internal: mem required  = 5809.34 MB (+ 1026.00 MB per state)
.
llama_init_from_file: kv self size  =  256.00 MB
AVX = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 1 | ARM_FMA = 1 | F16C = 0 | FP16_VA = 1 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 0 | VSX = 0 | 


In [8]:
output = llm("Q: How large is the earth's diameter? A: ", 
             max_tokens=20,  
             echo=True)
print(output["choices"])

[{'text': "Q: How large is the earth's diameter? A: 12,756 km B: 13,049 km C: ", 'index': 0, 'logprobs': None, 'finish_reason': 'length'}]



llama_print_timings:        load time =  2285.72 ms
llama_print_timings:      sample time =    14.11 ms /    20 runs   (    0.71 ms per token)
llama_print_timings: prompt eval time =  2285.67 ms /    15 tokens (  152.38 ms per token)
llama_print_timings:        eval time =  1244.78 ms /    19 runs   (   65.51 ms per token)
llama_print_timings:       total time =  3982.47 ms


In [33]:
retriever=vectordb.as_retriever(search_type="similarity")

In [34]:
from langchain.document_loaders import PyPDFLoader
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


In [47]:
!pip install llama-cpp-python==0.1.65 --force-reinstall --upgrade --no-cache-dir



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 4.2 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for typing-extensions>=4.5.0 from https://files.pythonhosted.org/packages/24/21/7d397a4b7934ff4028987914ac1044d3b7d52712f30e2ac7a2ae5bc86dd0/typing_extensions-4.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for numpy>=1.20.0 from https://files.pythonhosted.org/packages/2f/ac/be1f2767b7222347d2fefc18d8d58e9febfd9919190cc6fbd8a4d22d6eab/numpy-1.26.2-cp310-cp310-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.1 MB/s eta 0:00:00
  Obtaining dependency information for diskcache>=5.6.1 from https://files.pythonhosted.org/packages/3f/27/4570e78fc0bf5ea0ca45eb1de3818a23787af9b390c0b0a0033a1b8236f9/diskcache-5.6.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [48]:
from langchain.llms import LlamaCpp

LlamaCpp(model_path="llama.bin")

[autoreload of typing_extensions failed: Traceback (most recent call last):
  File "/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/langchain/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/langchain/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/Users/michieldekoninck/.pyenv/versions/3.10.6/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/langchain/lib/python3.10/site-packages/typing_extensions.py", line 733, in <module>
    class SupportsAbs(Protocol[T_c

ImportError: Could not import llama-cpp-python library. Please install the llama-cpp-python library to use this embedding model: pip install llama-cpp-python

In [50]:
import numpy as np
print(np.testing)

<module 'numpy.testing' from '/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/langchain/lib/python3.10/site-packages/numpy/testing/__init__.py'>


In [51]:
template = """{question}"""

QA_PROMPT = PromptTemplate(template=template, input_variables=["question"])
#QA_PROMPT = PromptTemplate(template=template, input_variables=["question"])


# Call OpenAI API via LangChain
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
#input_key="question",
def generate_response(query,chat_history):
    if query:
        my_qa = ConversationalRetrievalChain.from_llm(Llama(model_path="llama.bin"), retriever, QA_PROMPT, verbose=True, memory=memory)
        result = my_qa({"question": query, "chat_history": chat_history})

    return result["answer"]

In [52]:
generate_response('what is overfitting', '')

AttributeError: module 'llama_cpp.llama_cpp' has no attribute 'llama_init_from_file'

In [32]:
llm.create_chat_completion(
      messages = [
          {"role": "system", "content": "You are an assistant who perfectly describes images."},
          {
              "role": "user",
              "content": "Describe this image in detail please."
          }
      ]
)

Llama.generate: prefix-match hit

llama_print_timings:        load time =  2285.72 ms
llama_print_timings:      sample time =    18.58 ms /    26 runs   (    0.71 ms per token)
llama_print_timings: prompt eval time =  5677.37 ms /    30 tokens (  189.25 ms per token)
llama_print_timings:        eval time =  1490.82 ms /    25 runs   (   59.63 ms per token)
llama_print_timings:       total time =  7765.57 ms


{'id': 'chatcmpl-e3a66116-5f15-4db0-9ef6-cd40e2cfbdde',
 'object': 'chat.completion',
 'created': 1701437507,
 'model': 'llama.bin',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'This is a picture of a girl with long hair, she has a pink dress and a blue hat. '},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 31, 'completion_tokens': 26, 'total_tokens': 57}}

In [26]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

In [27]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [28]:
llm_chain = LLMChain(prompt=prompt, llm=llm)


ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

In [30]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])



In [31]:
llm = LlamaCpp(
    model_path="llama.bin",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

ImportError: Could not import llama-cpp-python library. Please install the llama-cpp-python library to use this embedding model: pip install llama-cpp-python

In [10]:
from langchain.chains import ConversationalRetrievalChain

In [24]:
chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever(), return_source_documents=True)

ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)